In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install openai
!pip install deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.1-py3-none-any.whl size=67316 sha256=cc43e0ccacc7348a6c44b693209caed3578905f0b3eb8e271be142abc1702e32
  Stored in directory: /root/.cache/pip/wheels/2f/9c/55/95d3609ccfc463eeffb96d50c756f1f1899453b85e92021a0a
Successfully built openai
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 17.7 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import os
import openai
import numpy as np
import pandas as pd
import glob
from deep_translator import GoogleTranslator
openai.api_key = 'api_key'

all_labels = [
    'Appeal_to_Authority',
    'Appeal_to_Popularity',
    'Appeal_to_Values',
    'Appeal_to_Fear-Prejudice',
    'Flag_Waving',
    'Causal_Oversimplification',
    'False_Dilemma-No_Choice',
    'Consequential_Oversimplification',
    'Straw_Man',
    'Red_Herring',
    'Whataboutism',
    'Slogans',
    'Appeal_to_Time',
    'Conversation_Killer',
    'Loaded_Language',
    'Repetition',
    'Exaggeration-Minimisation',
    'Obfuscation-Vagueness-Confusion',
    'Name_Calling-Labeling',
    'Doubt',
    'Guilt_by_Association',
    'Appeal_to_Hypocrisy',
    'Questioning_the_Reputation',
]

map_lang_data_trans = {
  'en': 'en',
  'fr': 'fr',
  'ge': 'de',
  'it': 'it',
  'po': 'pl',
  'ru': 'ru'
}

map_lang_data_text = {
  'en': 'English',
  'fr': 'French',
  'ge': 'German',
  'it': 'Italian',
  'po': 'Polish',
  'ru': 'Russian'
}

In [ ]:
def get_all_languages():
    return [x for x in os.listdir('./drive/MyDrive/Facultate/MasterAnul2/NLP/data/')]

def translate(src_lang, dest_lang, text):
   translated_text = GoogleTranslator(source=map_lang_data_trans[src_lang], target=map_lang_data_trans[dest_lang]).translate(text)
   return translated_text


def valid_text(text, labels, lang):
     text = text.lower().split()
     labels = labels.lower()
     print('text', text)
     print('labels', labels)

     special_word = translate('en', lang, 'persuasion')
     if special_word in text:
       return False

    
     special_word = translate('en', lang, 'technique')
     if special_word in text:
       return False

     special_word = translate('en', lang, 'techniques')
     if special_word in text:
       return False
   
     for l in labels.split(','):
        for label in l.replace('_', ' ').replace('-', ' ').split():
            if len(label) >= 5:
                translated_label = translate('en', lang, label)
                if (translated_label in text) or (label in text):
                  return False
    
     return True


def chatgpt_to_file(path, n_responses, lang):
  # 0.9 / 17 for under-represented labels, 0.10 / 6 for over-represented ones
  probabilities = [0.05294117647 for _ in range(len(all_labels))]
  probabilities[3] = 0.01666666666 # Appeal_to_Fear-Prejudice
  probabilities[14] = 0.01666666666 # Loaded_Language
  probabilities[16] = 0.01666666666 # Exaggeration-Minimisation
  probabilities[18] = 0.01666666666 # Name_Calling-Labeling
  probabilities[19] = 0.01666666666 # Doubt
  probabilities[22] = 0.01666666666 # Questioning_the_Reputation

  with open(path, 'a') as f:
      for _ in range(n_responses):
        n_labels = np.random.randint(low=1, high=(len(all_labels) // 5))
        random_labels = []
        original_labels = []

        for _ in range(n_labels):
          random_label = np.random.choice(all_labels, p=probabilities)
          if random_label not in original_labels: 
            original_labels.append(random_label)

            random_label = random_label.replace('_', ' ').replace('-', '/').lower()
            random_labels.append(random_label)

        original_labels = ','.join(original_labels)

        prompt_labels = ', '.join(random_labels)
        prompt = f"Generate a paragraph containing {prompt_labels} as persuasion techniques in {map_lang_data_text[lang]}" 
    

        response = openai.Completion.create(
          model="text-davinci-003",
          prompt=prompt,
          max_tokens=550,
          temperature=0.9
        )

        for choice in response['choices']:
            text = choice['text'].replace('\n', ' ')
            if valid_text(text, original_labels, lang):
              f.write(f"{text}\t{original_labels}\n")

In [ ]:
for lang in get_all_languages():
  for i in range(1, 11):
      chatgpt_to_file(f"./drive/MyDrive/Facultate/MasterAnul2/NLP/chatgpt_data/{lang}/laura_chatgpt"+ str(i)+ ".txt", 500, lang) 